# Generating TAPobjects

TAPobjects consist of three primary components, including the specification of the reactor, reactor species (gasses, inert gasses, and adspecies), and the reaction mechansim. Additional specifications of the TAPobject include defining what processes should be run (simulation, optimization, uncertainty, etc.), as well as other simulation details (mesh size, catalyst zone density, etc.). Although TAP objects can be generated from scratch, it can be much easier to use this example (as well as other jupyter notebooks provided here) to copy and make your own.

## Import TAPsolver

First, install the tapsolver package, where all of the tap object specifications can be found.

In [1]:
from tapsolver import *

## Define the reactor object

A new reactor object can be generated with the following commmand

In [2]:
# Defining a reactor object
new_reactor = reactor()

After defining a new reactor object, there are three reactor inputs that need to be defined based on your reactor of interest. These are defined in the form of dictionaries (specifying each of the reactor zones) and floats.

In [3]:
# The length of each zone (cm)
new_reactor.zone_lengths = {0:1.8675,1:0.2,2:1.8675}

# The void fraction of the packed material (zone 1 [inert], zone 2 [catalyst], zone 3[inert])
new_reactor.zone_voids = {0:0.4,1:0.4,2:0.4}

# The micro (TAP) reactor radius (cm)
new_reactor.reactor_radius = 0.19

## Define the reactor species object

A new reactor species object can be generated with the following commmand

In [4]:
# Defining a reactor species object
new_reactor_species = reactor_species()

After defining the reactor species object, some additional details can be adjusted based on your own experiment

In [5]:
# The reference diffusion coefficient in the inert zone (cm2/s)
new_reactor_species.inert_diffusion = 27

# The reference diffusion coefficient in the catalyst zone (cm2/s)
new_reactor_species.catalyst_diffusion = 27

# The reference temperature in the reactor (K)
new_reactor_species.reference_temperature = 700

# The mass of the species used for defining the inert and catalyst diffusion coefficient (a.m.u.)
new_reactor_species.reference_mass = 40

# The temperature of the current reactor
new_reactor_species.temperature = 700

## Add reactive gasses to the reactor species object

Once the reactor species object is defined, we can add reactive gasses with the following. First, we define a gas (in this case propane)

In [6]:
# Defining a gas species
C3H8 = define_gas()

Next, we specify the properties of the molecule and the pulse

In [7]:
# The mass of the current gas c3h8 (a.m.u.)
C3H8.mass = 44.1

# The # of molecules being pulsed into the reactor (nmol)
C3H8.intensity = 1.0

# The time delay of the gas being fed (seconds)
C3H8.delay = 0

An optional choice for the gas specification is noise and sigma, which represents the signal noise in the pulse and factor to scale the object function by (often equal to noise), respectively.

In [8]:
# Scaling factor of the objective function
C3H8.sigma = 0.1

# Noise to be included in the forward direction (i.e. simulation) (nmol/s)
C3H8.noise = 0.4

Finally, we add the new propane gas to the reactor species object

In [9]:
# Add the gas to the reactor species object
new_reactor_species.add_gas('C3H8',C3H8)

## Adding a new inert gas to the reactor species object 

Specifying an inert gas follows the same workflow

In [10]:
# Same specifications as c3h8
Ar = define_gas()
Ar.mass = 44.1
Ar.intensity = 1
Ar.delay = 0.0
Ar.noise = 0.01
Ar.sigma = 0.02

The only difference is the addition process

In [11]:
new_reactor_species.add_inert_gas('Ar',Ar)

## Adding a new adspecies to the reactor species object

Adspecies are added in a similar way to that of the gasses

In [12]:
# Define a new surface species
s = define_adspecies()

# Define the surface concentration (typically nmol/cm3)
s.concentration = 0

# Add the surface species to the reactor species object
new_reactor_species.add_adspecies('C3H8*',s)

And an active site can be added as 

In [13]:
# Same as for c3h8*
s = define_adspecies()
s.concentration = 120
new_reactor_species.add_adspecies('*',s)

## Defining the mechanism

The last major component to the TAPobject is the specification of the mechanism

In [15]:
new_mechanism = mechanism()

After defining the mechanism object, we can specify a kinetic process that we want to observe in the catalyst zone as follows.

In [16]:
new_mechanism.elementary_processes[0] = elementary_process('C3H8 + * <-> C3H8*')

For this mechanism, this is the first step ('elementary_processess[0]'). Additional steps can be added in a similar way, but with a change in the number associated with the process. We can then highlight the forward and reverse kinetic parameters for the reaction

In [17]:
new_mechanism.elementary_processes[0].forward.k = 10
new_mechanism.elementary_processes[0].backward.k = 5

The last steps for specifying the mechanism include highlighting which parameters should be included in the analysis. As will be noted in a later notebook, the mechanism can be defined in terms of free energy values or the arrhenius equation, so defining which to actually include is a necessary step.

In [22]:
# Specify which parameters we want to use in the analysis
for j in new_mechanism.elementary_processes:
    new_mechanism.elementary_processes[j].forward.use = 'k'
    try:
        new_mechanism.elementary_processes[j].backward.use = 'k'
    except:
        pass

Last, we "compile" the mechanism, generating all of the necessary inputs for the details to work with the FEniCS code. 

In [28]:
mechanism_constructor(new_mechanism)

# Display the reactants and stoichiometric matrix
print(new_mechanism.reactants)
print(new_mechanism.rate_array)

['C3H8', 'C3H8*', '*']
[[-1.  1. -1.]]


## Finalizing the TAPobject (Bringing it all together)

The last step is to merge the previously generated objects into a single TAP object

In [30]:
# Defining a general TAPobject
tap_1 = TAPobject()

# Adding the previously specified objects to the TAPobject
tap_1.mechanism = new_mechanism
tap_1.reactor_species = new_reactor_species
tap_1.reactor = new_reactor

## Storing the TAPobject

It is often easiest to store and load the TAPobject for later use. The TAPobject can be stored with the following command.

In [38]:
# Store the TAPobject 
save_object(tap_1,'./example_TAPobject.json')

The TAPobject can then be loaded with:

In [39]:
# Read the TAPobject
read_tap_1 = read_TAPobject('./example_TAPobject.json')

# Test display
print(read_tap_1.mechanism.reactants)
print(read_tap_1.mechanism.rate_array)

Gas already defined in dictionary.
Gas already defined in dictionary.
Gas already defined in dictionary.
['C3H8', 'C3H8*', '*']
[[-1.  1. -1.]]
